<a href="https://colab.research.google.com/github/ChristianIshimwe7/db-prediction-pipeline/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load pre-trained model (from Intro to ML)
model = joblib.load("heart_disease_model.pkl")
scaler = joblib.load("scaler.pkl")

# Feature order (must match training)
FEATURES = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
            'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']

def fetch_latest():
    response = requests.get("http://localhost:8000/patients/latest")
    if response.status_code != 200:
        raise Exception("Failed to fetch latest patient")
    return response.json()

def preprocess(data):
    df = pd.DataFrame([data])[FEATURES]
    df = df.fillna(df.median())
    X = scaler.transform(df)
    return X

def predict():
    patient = fetch_latest()
    X = preprocess(patient)
    prob = model.predict_proba(X)[0][1]
    pred = int(prob > 0.5)

    # Log prediction
    log = {
        "patient_id": patient['patient_id'],
        "prediction": pred,
        "probability": round(prob, 4),
        "model_version": "v1.0"
    }
    requests.post("http://localhost:8000/predictions/", json=log)
    print(f"Prediction: {pred} (Prob: {prob:.2f}) → Logged.")

if __name__ == "__main__":
    predict()

FileNotFoundError: [Errno 2] No such file or directory: 'heart_disease_model.pkl'